In [2]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import geowrangler.raster_zonal_stats as rzs
import geopandas as gpd

from pathlib import Path

/home/abbymoreno/miniconda3/envs/lacuna-fund2/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
# data files
DATA_DIR = Path("../../../data/")
WP_DIR = DATA_DIR / "02-raw" / "worldpop"

# population count
POP_COUNT = WP_DIR / "unconstrained_phl_ppp_2020.tif"
POP_COUNT_UNADJ = WP_DIR / "phl_ppp_2020_UNadj.tif"
# population density
POP_D = WP_DIR / "phl_pd_2020_1km.tif"
POP_D_UNADJ = WP_DIR / "phl_pd_2020_1km_UNadj.tif"

ADMIN_BOUNDS = DATA_DIR / "01-admin-bounds" / "target_admin_bounds.shp"

### Load AOI

In [4]:
aoi = gpd.read_file(ADMIN_BOUNDS)

In [5]:
# open rasterio
with rasterio.open(POP_COUNT) as src:
    data = src.read(1)
    data_crs = src.crs
    data_bounds = src.bounds

print(data.shape, data_crs, data_bounds)

(19781, 11613) EPSG:4326 BoundingBox(left=116.927916214, bottom=4.586250183000001, right=126.605416175, top=21.070416784)


## Population Count

In [6]:
%%time
pop_count_stats = rzs.create_raster_zonal_stats(
    aoi,
    POP_COUNT,
    aggregation=dict(
        func=["sum", "count", "mean", "median", "std", "min", "max"],
        column="population",
        output=[
            "population_count",
            "samples",
            "pop_mean",
            "pop_median",
            "pop_stdev",
            "pop_min",
            "pop_max",
        ],
    ),
    extra_args=dict(nodata=-99999),
)

CPU times: user 7.81 s, sys: 292 ms, total: 8.1 s
Wall time: 8.11 s


In [7]:
pop_count_stats.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,pop_min,pop_max,pop_mean,samples,population_count,pop_stdev,pop_median
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",10.708253,134.088531,19.414621,54,1048.389526,16.566447,16.313622
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",49.934814,171.413712,94.131187,85,8001.150879,23.749223,90.292122
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",16.287024,842.690857,164.335341,190,31223.714844,209.994516,61.385437
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044...",58.865456,158.769135,104.896740,21,2202.831543,25.539194,109.470459
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",0.835072,26.618620,5.960308,773,4607.318359,4.582612,4.241848


In [8]:
%%time
pop_count_unadj = rzs.create_raster_zonal_stats(
    aoi,
    POP_COUNT_UNADJ,
    aggregation=dict(
        func=["sum", "count", "mean", "median", "std", "min", "max"],
        column="population",
        output=[
            "population_count",
            "samples",
            "pop_mean",
            "pop_median",
            "pop_stdev",
            "pop_min",
            "pop_max",
        ],
    ),
    extra_args=dict(nodata=-99999),
)

CPU times: user 1min 53s, sys: 1.51 s, total: 1min 55s
Wall time: 1min 55s


In [9]:
pop_count_unadj.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,pop_min,pop_max,pop_mean,samples,population_count,pop_stdev,pop_median
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",10.648674,133.342484,19.306602,54,1042.556519,16.474274,16.222855
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",49.656986,170.460007,93.607462,85,7956.634277,23.617087,89.789757
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",16.196405,838.002319,163.421001,190,31049.990234,208.826150,61.043900
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044...",58.537941,157.885773,104.313105,21,2190.575195,25.397099,108.861389
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",0.830425,26.470520,5.927146,773,4581.684082,4.557115,4.218247


## Population Density

In [10]:
pop_d = rzs.create_raster_zonal_stats(
    aoi,
    POP_D,
    aggregation=dict(
        func=["sum", "count", "mean", "median", "std", "min", "max"],
        column="population",
        output=[
            "population_count",
            "samples",
            "pop_mean",
            "pop_median",
            "pop_stdev",
            "pop_min",
            "pop_max",
        ],
    ),
    extra_args=dict(nodata=-99999),
)

In [11]:
pop_d.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,pop_min,pop_max,pop_mean,samples,population_count,pop_stdev,pop_median
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1291.157593,1291.157593,1291.157593,1,1291.157593,0.000000,1291.157593
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",8300.595703,8300.595703,8300.595703,1,8300.595703,0.000000,8300.595703
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",612.800476,17503.455078,9460.138672,4,37840.554688,6034.533794,9862.148438
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044...",NaN,NaN,NaN,0,NaN,NaN,NaN
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",369.137238,863.373108,616.661316,8,4933.290527,173.546017,601.034546


In [13]:
pop_d_unadj = rzs.create_raster_zonal_stats(
    aoi,
    POP_D_UNADJ,
    aggregation=dict(
        func=["sum", "count", "mean", "median", "std", "min", "max"],
        column="population",
        output=[
            "population_count",
            "samples",
            "pop_mean",
            "pop_median",
            "pop_stdev",
            "pop_min",
            "pop_max",
        ],
    ),
    extra_args=dict(nodata=-99999),
)

In [14]:
pop_d_unadj

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,pop_min,pop_max,pop_mean,samples,population_count,pop_stdev,pop_median
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1283.973877,1283.973877,1283.973877,1,1283.973877,0.000000,1283.973877
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",8254.413086,8254.413086,8254.413086,1,8254.413086,0.000000,8254.413086
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",609.390930,17406.070312,9407.503906,4,37630.015625,6000.959202,9807.277344
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044...",NaN,NaN,NaN,0,NaN,NaN,NaN
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",367.083405,858.569519,613.230347,8,4905.842773,172.580446,597.690552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Namayan,PH137401018,"POLYGON ((121.02328 14.58135, 121.02309 14.581...",17389.779297,17389.779297,17389.779297,1,17389.779297,0.000000,17389.779297
875,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Plainview,PH137401022,"POLYGON ((121.03657 14.57933, 121.03734 14.579...",43431.089844,43431.089844,43431.089844,1,43431.089844,0.000000,43431.089844
876,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Navotas West,PH137503007,"MULTIPOLYGON (((120.94767 14.65256, 120.94768 ...",NaN,NaN,NaN,0,NaN,NaN,NaN
877,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Tanza,PH137503014,"POLYGON ((120.91841 14.71296, 120.92356 14.708...",263.541107,30310.074219,9175.870536,7,64231.093750,10689.164176,2609.654053
